# WMIR practice lesson on Spacy

### Obiettivi
- Usare il framework Spacy per estrarre informazioni rilevanti dalle frasi per arricchire le loro rappresentazioni.
- Addestra diversi modelli e confrontali con e senza la rappresentazioni "arricchita"  

#### Author
Claudiu Daniel Hromei, April 2023.  
hromei@ing.uniroma2.it

# Introduzione

[SpaCy](spacy.io) è una libreria open-source gratuita per il Natural Language Processing (NLP) in Python. Può essere usata per costruire sistemi di Information Extractrion o di Comprensione del Linguaggio Naturale, o anche per fare preprocessamento di testo per il Deep Learning.  
Alcune caratteristiche di SpaCy sono:
- **Tokenization**: suddividere il testo in unità indivisibili (**token**), formate dalle singole parole, segni di punteggiatura, simboli come 35% ecc.
- **Part-of-speech (POS) Tagging**: Assegnare i tipi delle parole (nome, verbo) ai token.
- **Dependency Parsing**: Assegnare delle etichette di dipendenza sintattica che descrivono le relazioni tra i diversi token. Ad esempio un token è oggetto mentre un altro è il soggetto.
- **Lemmatization**: Convertire le parole nella loro forma base, in italiano nel lemma. Ad esempio la forma all'infinto dei verbi oppure la forma maschile singolare dei nomi. In inglese, il lemma di “was” è “be”, e il lemma di “rats” è “rat”.
- **Sentence Boundary Detection (SBD)**: Trovare e suddividere i testi nelle diverse frasi.
- **Named Entity Recognition (NER)**: Trovare ed etichettare gli oggetti del mondo reale che hanno un nome, ad esempio persone, compagnie o luoghi. "New York City", "Giovanni" sono delle Named Entities.
- **Entity Linking (EL)**: Disambiguare le entità testuali in modo da assegnargli degli identificatori univoci in una base di conoscenza.
- **Similarity**: Confrontare le parole, testi e documenti e fornire una misura di similarità tra loro.
- **Text Classification**: Assegnare delle categorie o etichette a interi documenti o parti di esso.
- **Rule-based Matching**: Trovare le sequenze di token in base alle loro annotazioni linguistiche, in modo simile alle Regular Expression.
- **Training**: Aggiornare e migliorare le predizioni di un modello statistico
- **Serialization**: Salvare gli oggetti su file o su stringhe di bytes.

# Required Libraries

In [4]:
import pandas as pd

from IPython.display import display, HTML

In [5]:
# option to print all the value of cells in DataFrames
pd.set_option("max_colwidth", None)

### Install spacy and download the english pipeline

In [6]:
# install the spacy module
%pip install spacy


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
import spacy
from spacy import displacy

# Esempio di annotazione
Vogliamo stampare le annotazioni sintattiche di una frase.

In [10]:
input_string = "In 1982, Mark drove his car from Los Angeles to Las Vegas until 5 of july"
nlp = spacy.load('en_core_web_sm')

In [12]:
def print_annotation(input_string):
    # esegue il modello del linguaggio sulla stringa in input
    # dal doc possiamo estrarre molte cose, come le frasi, le singole parole, le relazioni ecc.
    doc = nlp(input_string)

    words = []
    # per ogni frase del documento
    for sent in doc.sents:
        # per ogni parola di indice i della frase
        for i, word in enumerate(sent):
            if word.head is word:
                head_idx = 0
            else:
                head_idx = doc[i].head.i+1
            if head_idx == i + 1:
                head_idx = 0

            entity_tag = word.ent_type_
            if len(entity_tag) == 0:
                entity_tag = "O"
        
            word_obj = {
                "id": str(i+1), 
                "word": str(word), 
                "lemma": word.lemma_, 
                "tag": word.tag_, 
                "pos": word.pos_,
                "entity": entity_tag,
                "dependency": word.dep_, 
                "head_id": str(head_idx)
            }
            words.append(word_obj)

    df = pd.DataFrame(words, columns=["id", "word", "lemma", "tag", "pos", "entity", "dependency", "head_id"])
    display(df) # stampiamo tutte le dipendenze tra parole, il lemma, il tag, il tipo di entità

- O: significa other. Nella colonna entity vuol dire che non fa parte di nessuna entità;
- Root: nella colonna dependency vuol dire che non ha nessuna dipendenza in entrata;

In [13]:
print_annotation(input_string)

,id,word,lemma,tag,pos,entity,dependency,head_id
0,1,In,in,IN,ADP,O,prep,5
1,2,1982,1982,CD,NUM,DATE,pobj,1
2,3,",",",",",",PUNCT,O,punct,5
3,4,Mark,Mark,NNP,PROPN,PERSON,nsubj,5
4,5,drove,drive,VBD,VERB,O,ROOT,0
5,6,his,his,PRP$,PRON,O,poss,7
6,7,car,car,NN,NOUN,O,dobj,5
7,8,from,from,IN,ADP,O,prep,5
8,9,Los,Los,NNP,PROPN,GPE,compound,10
9,10,Angeles,Angeles,NNP,PROPN,GPE,pobj,8


In [23]:
# Giacomo: mia aggiunta per capire esercizio 3.
sentence_2_verbs_by_giacomo = "John was eating an apple, but the apple fell to the ground."
print_annotation(sentence_2_verbs_by_giacomo)

,id,word,lemma,tag,pos,entity,dependency,head_id
0,1,John,John,NNP,PROPN,PERSON,nsubj,3
1,2,was,be,VBD,AUX,O,aux,3
2,3,eating,eat,VBG,VERB,O,ROOT,0
3,4,an,an,DT,DET,O,det,5
4,5,apple,apple,NN,NOUN,O,dobj,3
5,6,",",",",",",PUNCT,O,punct,3
6,7,but,but,CC,CCONJ,O,cc,3
7,8,the,the,DT,DET,O,det,9
8,9,apple,apple,NN,NOUN,O,nsubj,10
9,10,fell,fall,VBD,VERB,O,conj,3


In [25]:
sentence_without_proper_name = "The thing is we do not have a choice!"
print_annotation(sentence_without_proper_name)

,id,word,lemma,tag,pos,entity,dependency,head_id
0,1,The,the,DT,DET,O,det,2
1,2,thing,thing,NN,NOUN,O,nsubj,3
2,3,is,be,VBZ,AUX,O,ROOT,0
3,4,we,we,PRP,PRON,O,nsubj,7
4,5,do,do,VBP,AUX,O,aux,7
5,6,not,not,RB,PART,O,neg,7
6,7,have,have,VB,VERB,O,ccomp,3
7,8,a,a,DT,DET,O,det,9
8,9,choice,choice,NN,NOUN,O,dobj,7
9,10,!,!,.,PUNCT,O,punct,3


In [14]:
def visualize_annotation(input_string, style="dep"):
    doc = nlp(input_string)
    # Lo stile può essere "dep" per visualizzare le dipendenze tra le parole o "ent" per visualizzare le named entities
    displacy.render(doc, style=style, jupyter=True, options={"distance": 140}) # aumenta la distance per migliorare la leggibilità (default 140)

In [15]:
visualize_annotation(input_string, style="dep")

In [16]:
visualize_annotation(input_string, style="ent") # GPE = Global Position Entity

# Information extraction

Il seguente metodo permette di ottenere informazioni su una singola parola della frase in input.

In [13]:
def get_word_annotation(input_string, word_string):
    doc = nlp(input_string)
    
    words = []
    for sent in doc.sents:
        for i, word in enumerate(sent):
            if word.head is word:
                head_idx = 0
            else:
                head_idx = doc[i].head.i+1
            if head_idx == i + 1:
                head_idx = 0

            entity_tag = word.ent_type_
            if len(entity_tag) == 0:
                entity_tag = "O"
            
            word_obj = {
                "id": i+1,
                "word": str(word), 
                "lemma": word.lemma_, 
                "tag": word.tag_, 
                "entity": entity_tag,
                "dependency": word.dep_, 
                "head id": head_idx
            }
            words.append(word_obj)
    
    for word in words:
        if word["word"] == word_string:
            return word
    
    return None

In [14]:
print(get_word_annotation(input_string, "Mark"))

{'id': 4, 'word': 'Mark', 'lemma': 'Mark', 'tag': 'NNP', 'entity': 'PERSON', 'dependency': 'nsubj', 'head id': 5}


### Esercizio 1: Trovare le relazioni (dependencies)

Definisci un metodo che prende in input una frase (`input_string`) e il nome di una relazione (`relation_string`), analizza l'input con spacy e restituisce le triple (w1, w2, relation) dove w è un lemma. Se la relazione non p presente, restituisce un array vuoto.

```python
def search_relation(input_string, relation_string):
    # your code here...
    return word_obj_list
```

In [76]:
from typing import Callable, Dict, List
from spacy.tokens import Token

class Word:
    def __init__(self, word: Token, id: int, head_idx: int, entity_tag: str):        
        self.id = id
        self.word = str(word)
        self.lemma = word.lemma_
        self.tag = word.tag_
        self.pos = word.pos_
        self.entity = entity_tag
        self.dependency = word.dep_
        self.head_id = str(head_idx)
    
    # rappresentazione leggibile dell'oggetto (per utenti)
    def __str__(self):
        return "Word [" + \
            f"id: {self.id}, " +  \
            f"word: {self.word}, " +  \
            f"lemma: {self.lemma}, " +  \
            f"tag: {self.tag}, " + \
            f"pos: {self.pos}, " + \
            f"entity: {self.entity}, " +  \
            f"dependency: {self.dependency}, " +  \
            f"head_id: {self.head_id}" + \
        "]"   
        

    # rapprsentazione non ambigua dell'oggetto (per sviluppatori)
    def __repr__(self):
        return self.word  

# questa parte di codice la riutilizzo anche dopo: data una frase restituisce la lista di parole con tutte le informazioni utili di spacy
def _word_data(sentence: str, is_dict: bool):
    doc = nlp(sentence)
    if is_dict:
        words: Dict[int, Word] = {}
    else:
        words: List[Word] = []
    for sent in doc.sents:
        for i, word in enumerate(sent):
            if word.head is word:
                head_idx = 0
            else:
                head_idx = doc[i].head.i+1
            if head_idx == i + 1:
                head_idx = 0

            entity_tag = word.ent_type_
            if len(entity_tag) == 0:
                entity_tag = "O"

            word_obj = Word(word, i+1, head_idx, entity_tag)
            
            if is_dict:
                words[word_obj.id] = word_obj
            else:
                words.append(word_obj)
    return words

def get_word_data_list(sentence: str) -> List[Word]:
    return _word_data(sentence=sentence, is_dict=False)

def get_word_data_dict(sentence: str) -> Dict[int, Word]:
    return _word_data(sentence=sentence, is_dict=True)

In [77]:
def search_relation(input_string, relation_string):
    words: List[Word] = get_word_data_list(input_string)

    word_obj_set = set()
    for word_obj in words:
        w1 = word_obj.word
        dep = word_obj.dependency
        
        # trova la parola collegata dall'id
        head_id = word_obj.head_id
        w2 = "-"
        for word_obj in words:
            if head_id == word_obj.id:
                w2 = word_obj.word
                break

        # evito le relazioni duplicate
        if (head_id, id, dep) not in word_obj_set and relation_string == dep:
            word_obj_set.add((w1, w2, dep))
    
    return list(word_obj_set)

Ad esempio, cerchiamo tutte le parole in cui una è preposizione dell'altra

In [78]:
search_relation(input_string, "prep")

[('to', '-', 'prep'),
 ('In', '-', 'prep'),
 ('from', '-', 'prep'),
 ('until', '-', 'prep'),
 ('of', '-', 'prep')]

### Esercizio 2: Trovare le entità
Definire un metodo che prende in input una frase (`input_string`) e il nome di un tipo di entità (`entity_type_string`), analizza con spacy l'input e restituisce le parole (cioè gli `objects`, non le stringhe) descritti da quella entità. Se il tipo dell'entità non è presente, restituisce un array vuoto.

```python
def search_entity(input_string, entity_type_string):
    return word_obj_list
```

In [79]:
# Vanno restituiti gli object (e.g. "Los Angeles"), non le singole stringhe "Los" e "Angeles"

def search_entity(input_string, ent_label):
    entities_list = []
    doc = nlp(input_string)
    for ent in doc.ents:
        if ent.label_ == ent_label:
            entities_list.append((ent.text, ent.label_))
    return entities_list

In [80]:
entities = search_entity(input_string, "GPE") # GPE | DATE
print(entities)

[('Los Angeles', 'GPE'), ('Las Vegas', 'GPE')]


### Esercizio 3: Arricchire le frasi
Per ogni frase nel dataset QuestionClassification, estrai le relazioni `subject-verb` e `verb-object`. Aggiungi queste coppie all'input originale, divise dal simbolo `#`. Ad esempio:

- Frase: '*What is the full form of .com?*' 
- `subject-verb`: *What is*  
- `verb-object`: *is the full form* 
- Frase arricchita: '*What is the full form of .com? # What is # is the full form*'  

Salva le frasi arricchite in un nuovo dataframe, addestra un classificatore (SVM, NB, Rocchio..) e valutalo.

In [178]:
def enrich_sentence(input_sentence: str) -> str:
    words: List[Word] = get_word_data_list(input_sentence)
    subjects = find_subject(words)
    objects = find_object(words)

    words_dict: Dict[int, Word] = get_word_data_dict(input_sentence)
    result_str = input_sentence
    
    # aggiungo subject-verb
    for sub in subjects:
        # trovo il verbo
        verb = words_dict[int(sub.head_id)]

        # trovo eventuali parole collegate
        sub_words = []
        for word in words:
            if word.id == int(word.head_id):
                sub_words.append(word)
        # riordino
        sub_words.append(sub)
        sub_words.sort(key=lambda w: w.id)
        complete_sub = ""
        # traduco in stringa
        for sw in sub_words:
            complete_sub += sw.word + " "

        result_str += f" ~ {complete_sub}{verb.word}"
    
    # soggetto sottinteso
    if len(subjects) == 0:
        result_str += " ~ You"


    # aggiungo verb-object
    for obj in objects:
        # trovo il verbo
        verb = words_dict.get(int(obj.head_id))

        # trovo eventuali parole collegate
        obj_words = []
        for word in words:
            if obj.id == int(word.head_id):
                obj_words.append(word)
        obj_words.append(obj)
        # riordino
        obj_words.sort(key=lambda w: w.id)
        complete_obj = ""
        # traduco in stringa
        l = len(obj_words)
        for i, ow in enumerate(obj_words):
            complete_obj += ow.word
            if l != i + 1:
                complete_obj += " "

        if verb != None:
            result_str += f" # {verb.word} {complete_obj}"
        else:
            result_str += f" # {complete_obj}"

    return result_str

    

def find_subject(words: List[Word]) -> List[Word]:
    possible_subjects = ["attr", "advmod"]
    return list(filter(lambda w: (w.dependency in possible_subjects and (w.id == 1)) or w.word == "Who",  words))
    


def find_object(words: List[Word]) -> List[Word]:
    possible_objects = ["nsubj", "dobj", "dative", "oprd"]
    return list(filter(lambda w: w.dependency in possible_objects and w.word != "Who",  words))
    
def find_verb(words: List[Word]) -> List[Word]:
    possible_verbs = ["ROOT", "relcl"]
    return list(filter(lambda w: w.dependency in possible_verbs,  words))

In [166]:
sents = [
    "What is the full form of .com?",
    "Where is the Orinoco ?",
    "What is the best way to overcome a fear ?",
    "Name a South African diamond producer ?" ,
    "Why do heavier objects travel downhill faster ?", 
    "Who made the musical plea Be True to Your School ?"
]

for s in sents:
    print(enrich_sentence(s))

print_annotation(sents[5])

What is the full form of .com? ~ What is # is the full form of
Where is the Orinoco ? ~ Where is # is the Orinoco
What is the best way to overcome a fear ? ~ What is # is the best way overcome # overcome a fear
Name a South African diamond producer ? ~ You # Name a African diamond producer
Why do heavier objects travel downhill faster ? ~ Why travel # travel heavier objects
Who made the musical plea Be True to Your School ? ~ Who made # Be the musical plea


,id,word,lemma,tag,pos,entity,dependency,head_id
0,1,Who,who,WP,PRON,O,nsubj,2
1,2,made,make,VBD,VERB,O,ROOT,0
2,3,the,the,DT,DET,O,det,5
3,4,musical,musical,JJ,ADJ,O,amod,5
4,5,plea,plea,NN,NOUN,O,nsubj,6
5,6,Be,be,VB,AUX,O,ccomp,2
6,7,True,true,JJ,ADJ,O,acomp,6
7,8,to,to,IN,ADP,O,prep,7
8,9,Your,your,PRP$,PRON,O,poss,10
9,10,School,school,NN,NOUN,O,pobj,8


In [181]:
# arricchiamo il dataset (ci vuole un po'!)
from sklearn.feature_extraction.text import TfidfVectorizer

# NON E' PERFETTO, ma qualcosa fa (Impiega 52s sul mio PC)
def enrich(df) -> pd.DataFrame:
    data_dict = df.to_dict()
    questions : Dict[int, str] = data_dict["questions"]
    c = 0
    items = questions.items()
    l = len(items)
    for i, question in items:
        sentence_enriched = enrich_sentence(question)

        question = data_dict["questions"][i]

        if question != None:
            data_dict["questions"][i] = sentence_enriched
        else:
            f"question with id {i} skipped"
        
        c += 1
        print(f" {c}/{l} - {sentence_enriched}")
    return pd.DataFrame(data_dict)


training_data = pd.read_csv("./Dataset04svm/train.csv")
testing_data = pd.read_csv("./Dataset04svm/test.csv")


enriched_training_data = enrich(training_data)
enriched_testing_data = enrich(testing_data)


 1/5452 - How did serfdom develop in and then leave Russia ? ~ How develop # develop serfdom # leave Russia
 2/5452 - What films featured the character Popeye Doyle ? ~ You # featured What films # featured the character Doyle
 3/5452 - How can I find a list of celebrities ' real names ? ~ How find # find I # find a list of
 4/5452 - What fowl grabs the spotlight after the Chinese Year of the Monkey ? ~ You # grabs What fowl # grabs the spotlight
 5/5452 - What is the full form of .com ? ~ What is # is the full form of
 6/5452 - What contemptible scoundrel stole the cork from my lunch ? ~ You # stole What contemptible scoundrel # stole the cork
 7/5452 - What team did baseball 's St. Louis Browns become ? ~ You # did What team # become baseball Louis Browns
 8/5452 - What is the oldest profession ? ~ What is # is the oldest profession
 9/5452 - What are liver enzymes ? ~ What are # are liver enzymes
 10/5452 - Name the scar-faced bounty hunter of The Old West . ~ You # Name the faced bo

In [185]:
enriched_training_data.to_csv(path_or_buf="./Dataset04svm/train_enriched.csv", index=False)
enriched_testing_data.to_csv(path_or_buf="./Dataset04svm/test_enriched.csv", index=False)

In [186]:
# feature extraction
vectorizer_enriched = TfidfVectorizer()

# fit: Estraiamo le feature tf-idf dal dataset
# transform: convertiamo le frasi del dataset in vettori tf-idf
X_train = vectorizer_enriched.fit_transform(enriched_training_data['questions'].tolist()) # matrice sparsa del training set. 
# transform: qui convertiamo solamente le frasi del dataset
X_test = vectorizer_enriched.transform(enriched_testing_data['questions'].tolist()) # e del testing set
y_train = enriched_training_data['classes'].tolist()
y_test = enriched_testing_data['classes'].tolist()

In [188]:
# usiamo il modello migliore SVM per tf-idf trovato nella lezione 5

from sklearn.metrics import classification_report
from sklearn.svm import SVC


svm = SVC(C=1.0, kernel='linear', degree=0, gamma=0.01, max_iter=-1, decision_function_shape='ovo')

svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print(classification_report(y_pred, y_test, target_names=['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']))

              precision    recall  f1-score   support

        ABBR       0.78      1.00      0.88         7
        DESC       0.97      0.83      0.90       161
        ENTY       0.72      0.80      0.76        85
         HUM       0.92      0.86      0.89        70
         LOC       0.83      0.88      0.85        76
         NUM       0.88      0.98      0.93       101

    accuracy                           0.87       500
   macro avg       0.85      0.89      0.87       500
weighted avg       0.88      0.87      0.87       500



Migliora di 5 punti percentuali!!!

### Esercizio 4: Sostituisci i testi con le entità
Per ogni frase nel dataset QuestionClassification, estrai le entità annotate dal modulo spacy solo per i nomi propri, di città e le date (`PROPN`, `GPE`, `DATE`) e sostituisci le porzioni di testo corrispondenti con il nome dell'entità. Ad esempio:

- Frase iniziale  : '*In 1982, Mark drove his car from Los Angeles to Las Vegas until 5 of july*'  
- Frase modificata: '*In 1982, PERSON drove his car from GPE to GPE until DATE*'

Salva le frasi arricchite in un nuovo dataframe, addestra un classificatore (SVM, NB, Rocchio..) e valutalo.

**WARNING**: fare attenzione ai `compounds`, dovrebbero essere sostituiti da una singola entità. Per esempio: *Las Vegas* => `GPE`

In [207]:
def find_named_entities(sentence: str):
    entity_labels = nlp.get_pipe('ner').labels
    entity_found = []
    for entity_label in entity_labels:
        ents = search_entity(sentence, entity_label)
        if len(ents) != 0:
            entity_found.append(ents)
    return entity_found

def replace_named_entities(sentence: str):
    entities_to_replace = find_named_entities(sentence)
    for e in entities_to_replace:
        print(f"sostituisco {e[0][1]} al posto di {e[0][0]}")
        sentence = sentence.replace(e[0][0], e[0][1])
    return sentence

# NON E' PERFETTO, ma qualcosa fa (Impiega 8 min sul mio PC)
def replace_dataset(df) -> pd.DataFrame:
    data_dict = df.to_dict()
    questions : Dict[int, str] = data_dict["questions"]
    c = 0
    items = questions.items()
    l = len(items)
    for i, question in items:
        sentence_enriched = replace_named_entities(question)

        question = data_dict["questions"][i]

        if question != None:
            data_dict["questions"][i] = sentence_enriched
        else:
            f"question with id {i} skipped"
        
        c += 1
        print(f" {c}/{l} - {sentence_enriched}")
    return pd.DataFrame(data_dict)

training_data = pd.read_csv("./Dataset04svm/train.csv")
testing_data = pd.read_csv("./Dataset04svm/test.csv")


replaced_training_data = replace_dataset(training_data)
replaced_testing_data = replace_dataset(testing_data)

sostituisco GPE al posto di Russia
 1/5452 - How did serfdom develop in and then leave GPE ?
sostituisco PERSON al posto di Popeye Doyle
 2/5452 - What films featured the character PERSON ?
 3/5452 - How can I find a list of celebrities ' real names ?
sostituisco EVENT al posto di the Chinese Year of the
sostituisco PRODUCT al posto di Monkey
 4/5452 - What fowl grabs the spotlight after EVENT PRODUCT ?
 5/5452 - What is the full form of .com ?
 6/5452 - What contemptible scoundrel stole the cork from my lunch ?
sostituisco FAC al posto di St. Louis Browns
 7/5452 - What team did baseball 's FAC become ?
 8/5452 - What is the oldest profession ?
 9/5452 - What are liver enzymes ?
sostituisco ORG al posto di The Old West
 10/5452 - Name the scar-faced bounty hunter of ORG .
sostituisco PERSON al posto di Ozzy Osbourne
 11/5452 - When was PERSON born ?
 12/5452 - Why do heavier objects travel downhill faster ?
sostituisco WORK_OF_ART al posto di The Pride of the Yankees
 13/5452 - Who wa

In [208]:
replaced_training_data.to_csv(path_or_buf="./Dataset04svm/train_replaced.csv", index=False)
replaced_testing_data.to_csv(path_or_buf="./Dataset04svm/test_replaced.csv", index=False)

In [212]:
# feature extraction
vectorizer_replaced = TfidfVectorizer()

# fit: Estraiamo le feature tf-idf dal dataset
# transform: convertiamo le frasi del dataset in vettori tf-idf
X_train2 = vectorizer_replaced.fit_transform(replaced_training_data['questions'].tolist()) # matrice sparsa del training set. 
# transform: qui convertiamo solamente le frasi del dataset
X_test2 = vectorizer_replaced.transform(replaced_testing_data['questions'].tolist()) # e del testing set
y_train2 = replaced_training_data['classes'].tolist()
y_test2 = replaced_testing_data['classes'].tolist()

In [213]:
# usiamo il modello migliore SVM per tf-idf trovato nella lezione 5

from sklearn.metrics import classification_report
from sklearn.svm import SVC


svm = SVC(C=1.0, kernel='linear', degree=0, gamma=0.01, max_iter=-1, decision_function_shape='ovo')

svm.fit(X_train2, y_train2)

y_pred2 = svm.predict(X_test2)

print(classification_report(y_pred2, y_test2, target_names=['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']))

              precision    recall  f1-score   support

        ABBR       0.78      1.00      0.88         7
        DESC       0.99      0.82      0.89       166
        ENTY       0.72      0.83      0.77        82
         HUM       0.89      0.89      0.89        65
         LOC       0.85      0.86      0.86        80
         NUM       0.86      0.97      0.91       100

    accuracy                           0.87       500
   macro avg       0.85      0.90      0.87       500
weighted avg       0.88      0.87      0.87       500



Ehm... anche qui 87%. Le risposte NUM hanno un f1-score minore rispetto al caso precedente

### Esercizio 5: Confronta i modelli
Confronta i modelli degli esercizi 3 e 4 con un modello che hai addestrato nelle precedenti lezioni in termini di F1 measure.

### Risposta

Sono entrambi equivalenti. Ma e' meglio il primo perché impiega molto meno tempo e produce risultati migliori nelle risposte NUM.